## Imports

In [1]:
import os, shutil
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from sqlalchemy import create_engine
from matplotlib import pyplot as plt
from datetime import datetime

#turn off annoying warnigns
pd.options.mode.chained_assignment = None  # default='warn'

## Task 1: Data Collection

In [2]:
url = "https://www.worldometers.info/coronavirus/"
htmlContent = requests.get(url).text

soup = BeautifulSoup(htmlContent, 'html.parser')
covidTable = soup.find("table",attrs={"id": "main_table_countries_today"})

## Task 2: Main Data Extract

In [5]:
# extract the table headings from the soup
head = covidTable.thead.find_all("tr")
headings = []

for th in head[0].find_all("th"):
    headings.append(th.text.replace("\n", "").strip())

# extract actual data from the soup
body = covidTable.tbody.find_all("tr")
data = []

# iterate through every row in the html
for r in range(1,len(body)):
    row = []
    # find all column entries in that particular row
    for tr in body[r].find_all("td"):
        row.append(tr.text.replace("\n","").strip())
    len(row)
    data.append(row)

df = pd.DataFrame(data,columns = headings)

######################################################################
#                         CLEAN DATA TABLE                           #
######################################################################

# define nan object
NaN = np.nan

# recover appropriate columns
df = df.filter(['Continent', 'Country,Other','TotalCases','TotalRecovered','Serious,Critical','ActiveCases','TotalDeaths']) 

# strip artefacts
df = df.replace(',','', regex=True)
df = df.replace('\+','', regex=True)
df = df.replace('', NaN, regex=True)

# convert appropriate columns to integer type
df[['TotalCases', 'TotalRecovered','Serious,Critical','ActiveCases','TotalDeaths']] = df[['TotalCases', 'TotalRecovered','Serious,Critical','ActiveCases','TotalDeaths']].apply(pd.to_numeric, downcast = 'float', errors = 'coerce')
df.head(20)

# add date of processing to all rows
now = datetime.now()
dt_string = now.strftime("%d%m%Y")
df['date_processed'] = dt_string

#rename the columns for consistency
df.rename(columns = {'Continent':'continent','Country,Other':'country','TotalCases':'total_cases','TotalRecovered':'total_recovered','Serious,Critical':'critical_active','ActiveCases':'active_cases','TotalDeaths':'total_deaths'}, inplace=True)


,continent,country,total_cases,total_recovered,critical_active,active_cases,total_deaths,date_processed
0,Asia,Asia,57595824.0,54674024.0,28217.0,2101041.0,820757.0,11072021
1,South America,South America,33957712.0,31158308.0,28360.0,1764338.0,1035064.0,11072021
2,Europe,Europe,48893548.0,45864416.0,6074.0,1916059.0,1113072.0,11072021
3,Africa,Africa,5971777.0,5185839.0,4429.0,634195.0,151743.0,11072021
4,Australia/Oceania,Oceania,81303.0,69944.0,47.0,10038.0,1321.0,11072021
...,...,...,...,...,...,...,...,...
224,Australia/Oceania,Marshall Islands,4.0,4.0,NaN,0.0,NaN,11072021
225,Australia/Oceania,Samoa,3.0,3.0,NaN,0.0,NaN,11072021
226,Africa,Saint Helena,2.0,2.0,NaN,0.0,NaN,11072021
227,Australia/Oceania,Micronesia,1.0,1.0,NaN,0.0,NaN,11072021


## Task 3: Extract Tables

In [9]:
############# world
world_df = df.loc[df['country'] == "World"]

# select our data for plotting:
world_df = world_df.filter(['total_cases','total_recovered','critical_active','active_cases','total_deaths', 'date_processed'])

# this is the correct way to add a new column from existing columns!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
world_df = world_df.assign(non_critical_active = world_df['active_cases'].astype(float) - world_df['critical_active'].astype(float))
# drop active_cases now
world_df = world_df.drop(columns = ['active_cases']).reset_index(drop=True)

# add in percentage columns
world_df = world_df.assign(total_recovered_percentage = (int(world_df['total_recovered'].astype(float)) / int(world_df['total_cases'].astype(float))) * 100)
world_df = world_df.assign(critical_active_percentage = (world_df['critical_active'].astype(float) / world_df['total_cases'].astype(float)) * 100)
world_df = world_df.assign(total_deaths_percentage = (world_df['total_deaths'].astype(float) / world_df['total_cases'].astype(float)) * 100)
world_df = world_df.assign(non_critical_active_percentage = (world_df['non_critical_active'].astype(float) / world_df['total_cases'].astype(float)) * 100)

############ continent 
continents_df = df.iloc[0:5]
continents_df.drop(columns=['country'], inplace=True)

# create a row of data to append to the continents for North America
na = df.loc[df['continent'] == "North America"]
del na["continent"]
na = na.replace('','0', regex=True)
row = []
row.append("North America")

for column in na.columns[1:]:
    na[column] = pd.to_numeric(na[column], errors='coerce')
    row.append(round(na[column].sum()))
row = [row]

continents_df = continents_df.append(pd.DataFrame( row, columns=continents_df.columns), ignore_index = True)

continents_df = continents_df.assign(non_critical_active = continents_df['active_cases'].astype(float) - continents_df['critical_active'].astype(float))
# drop active_cases now
continents_df = continents_df.drop(columns = ['active_cases']).reset_index(drop=True)

# add in percentage columns
continents_df = continents_df.assign(total_recovered_percentage = (continents_df['total_recovered'].astype(float) / continents_df['total_cases'].astype(float)) * 100)
continents_df = continents_df.assign(critical_active_percentage = (continents_df['critical_active'].astype(float) / continents_df['total_cases'].astype(float)) * 100)
continents_df = continents_df.assign(total_deaths_percentage = (continents_df['total_deaths'].astype(float) / continents_df['total_cases'].astype(float)) * 100)
continents_df = continents_df.assign(non_critical_active_percentage = (continents_df['non_critical_active'].astype(float) / continents_df['total_cases'].astype(float)) * 100)

continents_df = continents_df.sort_values(by=['total_cases'], ascending=False).reset_index(drop=True)

############ country
countries_df = df.loc[7:len(df)]
countries_df = countries_df.reset_index()
countries_df = countries_df.drop(columns = 'index')

countries_df = countries_df.assign(non_critical_active = countries_df['active_cases'].astype(float) - countries_df['critical_active'].astype(float))
countries_df = countries_df.drop(columns = ['active_cases']).reset_index(drop=True)


# add in percentage columns
countries_df = countries_df.assign(total_recovered_percentage = (countries_df['total_recovered'].astype(float) / countries_df['total_cases'].astype(float)) * 100)
countries_df = countries_df.assign(critical_active_percentage = (countries_df['critical_active'].astype(float) / countries_df['total_cases'].astype(float)) * 100)
countries_df = countries_df.assign(total_deaths_percentage = (countries_df['total_deaths'].astype(float) / countries_df['total_cases'].astype(float)) * 100)
countries_df = countries_df.assign(non_critical_active_percentage = (countries_df['non_critical_active'].astype(float) / countries_df['total_cases'].astype(float)) * 100)

countries_df = countries_df.sort_values(by=['total_cases'], ascending=False).reset_index(drop=True)

,continent,country,total_cases,total_recovered,critical_active,total_deaths,date_processed,non_critical_active,total_recovered_percentage,critical_active_percentage,total_deaths_percentage,non_critical_active_percentage
0,North America,USA,34726260.0,29234930.0,4340.0,622825.0,11072021,4864164.0,84.186808,0.012498,1.793527,14.007163
1,Asia,India,30871508.0,30000214.0,8944.0,408763.0,11072021,453586.0,97.177676,0.028972,1.324078,1.469271
2,South America,Brazil,19069004.0,17530368.0,8318.0,532949.0,11072021,997367.0,91.931220,0.043621,2.794844,5.230305
3,Europe,France,5808383.0,5645990.0,947.0,111321.0,11072021,50125.0,97.204162,0.016304,1.916557,0.862977
4,Europe,Russia,5783333.0,5200219.0,2300.0,143002.0,11072021,437812.0,89.917337,0.039769,2.472657,7.570237


## Task 4: Create CSVs

In [42]:
# create a folder for the csv data extracts if there isn't aready one using the os module
path = os.getcwd()
directory = 'csv_extracts'

if os.path.isdir(directory):
    print("Extracts directory found at '%s'" % (path+directory))
    path = os.path.join(path, directory)

    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        print("Removing old file '%s' at '%s'" % (filename, file_path))
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))
else:
    print('Extracts directory not found, creating csv extracts directory and files...')
    path = os.path.join(path, directory)
    os.mkdir(path)
    try:
        if os.path.isdir(directory):
            print("Directory '% s' successfully created" % directory)
    except Exception as e:
        print("Could not create directory '% s'. Reason: %s" % (directory, e))
        
# create csv files from extracts

# world df
csv_name = path + "/world_csv_" + dt_string + ".csv"
print(csv_name)
world_df.to_csv(csv_name)

# continents df
csv_name = path + "/continent_csv_" + dt_string + ".csv"
print(csv_name)
continents_df.to_csv(csv_name)

# countries df
csv_name = path + "/country_csv_" + dt_string + ".csv"
print(csv_name)
countries_df.to_csv(csv_name)


Extracts directory found at '/Users/markdunbar/Documents/Repositories/Python-Projects-and-Challenges/Python Covid Tracker/src/covid-task-schedulercsv_extracts'
Removing old file 'continent_csv_11072021.csv' at 'csv_extracts/continent_csv_11072021.csv'
Removing old file 'country_csv_11072021.csv' at 'csv_extracts/country_csv_11072021.csv'
Removing old file 'world_csv_11072021.csv' at 'csv_extracts/world_csv_11072021.csv'
/Users/markdunbar/Documents/Repositories/Python-Projects-and-Challenges/Python Covid Tracker/src/covid-task-scheduler/csv_extracts/world_csv_11072021.csv
/Users/markdunbar/Documents/Repositories/Python-Projects-and-Challenges/Python Covid Tracker/src/covid-task-scheduler/csv_extracts/continent_csv_11072021.csv
/Users/markdunbar/Documents/Repositories/Python-Projects-and-Challenges/Python Covid Tracker/src/covid-task-scheduler/csv_extracts/country_csv_11072021.csv


## Task 5: Upload data extracts to database

In [38]:
engine = create_engine('sqlite:///../application/site.db', echo=True)
sqlite_connection = engine.connect()

world_df.to_sql("world", sqlite_connection, if_exists='replace')

continents_df.to_sql("continents", sqlite_connection, if_exists='replace')

countries_df.to_sql("countries", sqlite_connection, if_exists='replace')

sqlite_connection.close()


2021-07-11 00:34:15,589 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-07-11 00:34:15,590 INFO sqlalchemy.engine.base.Engine ()
2021-07-11 00:34:15,594 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-07-11 00:34:15,595 INFO sqlalchemy.engine.base.Engine ()
2021-07-11 00:34:15,601 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("world")
2021-07-11 00:34:15,602 INFO sqlalchemy.engine.base.Engine ()
2021-07-11 00:34:15,607 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("world")
2021-07-11 00:34:15,608 INFO sqlalchemy.engine.base.Engine ()
2021-07-11 00:34:15,609 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2021-07-11 00:34:15,611 INFO sqlalchemy.engine.base.Engine ()
2021-07-11 00:34:15,615 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_xinfo("world")
2021-07-11 00:34:15,616 INFO sqlalchemy.engine.base.Eng